In [ ]:
## 사용할 모듈 미리 로딩
import pandas as pd
import requests
import matplotlib.pyplot as plt
import matplotlib as mpl
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import time
import platform
import numpy as np
import random
import os
import shutil
from PIL import Image
import collections

if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

from sqlalchemy.sql import text as Text
from sqlalchemy import create_engine
from sqlalchemy import engine